In [ ]:
from PIL import Image
import numpy as np

In [ ]:
import lzma
import pickle
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [ ]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

In [ ]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])
print(train_y)

## 開始 Tensorflow

In [ ]:
import tensorflow as tf
from tfdot import tfdot

# Multilayer Convolutional Network

In [ ]:
# 使用 gfile 來讀檔
from tensorflow.python.platform import gfile
# 讀入 graph_def
with gfile.FastGFile("mnist_simple.pb",'rb') as f:
    graph_def = tf.GraphDef()
    x = f.read()
    #print(x)
    graph_def.ParseFromString(x)

In [ ]:
# 使用之前存下來的模型
X, Y_, prediction, accuracy, train_step, keep_prob, init_op= tf.import_graph_def(graph_def, name="", 
        return_elements=["X:0", "Y_:0", "prediction:0", 
                         "accuracy:0", "Adam", 'dropout/keep_prob:0', "init"])

In [ ]:
sess = tf.InteractiveSession()
init_op.run()
tf.summary.scalar(accuracy.op.name, accuracy)
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("log1", graph=sess.graph)

In [ ]:
for i in range(5000):
    rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
    if i%250 == 0:       
        summary_str, validation_accuracy = sess.run([summary_op, accuracy],
                        {X: validation_X[:1000], 
                                       Y_: validation_Y[:1000], 
                                       keep_prob: 1.0 })
        summary_writer.add_summary(summary_str, i)
        print("step %d, validation accuracy: %g"%(i, validation_accuracy))
    train_step.run({X: train_X[rnd_idx], Y_: train_Y[rnd_idx], keep_prob: 0.5 })

In [ ]:
summary_writer.close()
sess.close()

run `tensorboard --logdir=log1` in terminal and open http://localhost:6006



In [ ]:
!tensorboard --logdir=log1

### 同時紀錄三種準確度

In [ ]:
sess = tf.InteractiveSession()
init_op.run()
acc_summary = tf.summary.scalar("accuracy", accuracy)
training_summary_writer = tf.summary.FileWriter("log2/training", graph=sess.graph)
validation_summary_writer = tf.summary.FileWriter("log2/validation", graph=sess.graph)
testing_summary_writer = tf.summary.FileWriter("log2/testing", graph=sess.graph)


In [ ]:
for i in range(5000):
    rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
    if i%50 == 0:       
        summary_str, training_acc = sess.run([acc_summary, accuracy],
                        {X: train_X[:1000], Y_: train_Y[:1000], keep_prob: 1.0 })
        training_summary_writer.add_summary(summary_str, i)
        summary_str, validation_acc = sess.run([acc_summary, accuracy],
                        {X: validation_X[:1000], Y_: validation_Y[:1000], keep_prob: 1.0 })
        validation_summary_writer.add_summary(summary_str, i)
        summary_str, testing_acc = sess.run([acc_summary, accuracy],
                        {X: test_X[:1000], Y_: test_Y[:1000], keep_prob: 1.0 })
        testing_summary_writer.add_summary(summary_str, i)
        if i%250==0:
            print("step %d, train: %g, validation: %g, test: %g"%(i, training_acc, 
                                                              validation_acc, testing_acc))
    train_step.run({X: train_X[rnd_idx], Y_: train_Y[rnd_idx], keep_prob: 0.5 })

In [ ]:
testing_summary_writer.close()
validation_summary_writer.close()
training_summary_writer.close()
sess.close()

In [ ]:
!tensorboard --logdir=log2

## Image

In [ ]:
# accuracy
accuracy = tf.Variable(0.2, name="xxx")
accuracy_ = tf.placeholder("float")
tf.summary.scalar("acc", accuracy)
update = tf.assign(accuracy, accuracy_)

# image
img_ = tf.placeholder("float", shape=[None, 1080, 1920, 3])
img = tf.Variable(tf.zeros([1, 1080,1920,3]))
tf.summary.image("img", img)
update_img = tf.assign(img, img_)

# merge all summary
summary_op = tf.summary.merge_all()


In [ ]:
# rendering the image
from math import pi, sin
a = np.zeros((1080,1920,3))
i_ = np.arange(1,64)
c = np.random.uniform(size=(63,3))

def color_arrows(t):
    t_ = (i_*7.15+t)
    x1 = (t_/3%1*1920).astype(np.int32)
    y1 = (np.sin(t_)*500+500).astype(np.int32)
    for i in range(63):
        a[y1[i]:y1[i]+80, x1[i]:x1[i]+80] = c[i]
    return a

In [ ]:
# tensorflow session
with tf.Session() as sess:
    with  tf.summary.FileWriter("log3", graph=sess.graph) as summary_writer:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            sess.run(update, feed_dict={accuracy_: i/100.0})
            color_arrows(i/100)       
            sess.run(update_img, feed_dict={img_: a[None, ...]})
            summary_writer.add_summary(summary_op.eval(), i)

In [ ]:
!tensorboard --logdir=log3